In [1]:
import glob
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
#from adjustText import adjust_text

In [4]:
zoom = pd.read_excel(r"C:\Users\Admin\Documents\Studio Code\customer_retention_data\final_zoom.xlsx", dtype={'SID': str, 'SID_str': str})
goiten = pd.read_excel(r"C:\Users\Admin\Documents\Studio Code\customer_retention_data\final_goiten.xlsx", dtype={'SID_8digit':str,'SID_5digit': str})
students = pd.read_excel(r"C:\Users\Admin\Documents\Studio Code\customer_retention_data\final_student_list.xlsx", dtype={'SID_8digit':str,'SID_5digit': str})
canvas = pd.read_excel(r"C:\Users\Admin\Documents\Studio Code\customer_retention_data\final_canvas.xlsx", dtype={'SID': str})


print(students.head())
print(zoom.head())
print(goiten.head())
print(canvas.head())

c:\Users\Admin\anaconda3\Lib\site-packages\openpyxl\worksheet\_read_only.py:79: UserWarning: Data Validation extension is not supported and will be removed
  for idx, row in parser.parse():


         SID SID_8digit SID_5digit                  Name Grade_actual  \
0  2001-4683   20014683      14683        Tran Khanh Chi         HM 6   
1  2001-3864   20013864      13864     Nguyen Ngoc Thien         HM 6   
2  2300-3836   23003836      03836         Đào Quỳnh Anh      Grade K   
3  2300-3836   23003836      03836         Dao Quynh Anh          NaN   
4  2100-0100   21000100      00100  Nguyễn Hoàng Trúc An      Grade K   

     Grade  EFL                                  Note MVA/MVSM concat  \
0     HM 6  NaN  Found in goiten, not in student list      NaN    NaN   
1     HM 6  NaN  Found in goiten, not in student list      NaN    NaN   
2  Grade K  NaN  Found in goiten, not in student list      NaN    NaN   
3  Grade 2  NaN                                   NaN      NaN    NaN   
4  Grade K  NaN  Found in goiten, not in student list      NaN    NaN   

   4 last digits Học sinh cùng lớp có chung 4 số cuối ID           last_join  \
0            NaN                          

# Zoom dataset cleaning 

In [14]:
#take only the necessary columns 
zoom = zoom[['SID','Topic', 'ID', 'Host name', 'Host email', 'Start time', 'End time',
       'Participants', 'Duration (minutes)', 'Name (original name)', 'Email',
       'Join time', 'Leave time', 'student_stay_duration', 'Guest', 
       'date', 'time', 'day_of_week', 'hour', 'stu_join_time',
       'stu_leave_time', 'SID_str', 'role']]

#drop duplicates
zoom = zoom.drop_duplicates(subset=['ID', 'SID', 'Join time', 'Leave time', 'Name (original name)', 'student_stay_duration'], keep='last')

In [ ]:
#get student_stay_percent: because total class durations are different, we will get the percent of their stay compared to that class duration
zoom['student_stay_percent'] = (zoom['student_stay_duration']/zoom['Duration (minutes)']) *100

#get is_present: whether that class is counted as Present (student stay more than 25% of class time) or Absent (<25%)
zoom['is_present'] = (zoom['student_stay_percent']>25).astype(int)

#get minutes_late & minutes leaving early
#turn into datetime formats
zoom['Start time'] = pd.to_datetime(zoom['Start time'])
zoom['End time'] = pd.to_datetime(zoom['End time'])
zoom['Join time'] = pd.to_datetime(zoom['Join time'])
zoom['Leave time'] = pd.to_datetime(zoom['Leave time'])

#get minutes late
zoom['minutes_late'] = round((zoom['Join time']-zoom['Start time']).dt.total_seconds()/60)

#get minutes leaving early
zoom['minutes_leave_early'] = round((zoom['End time']-zoom['Leave time']).dt.total_seconds()/60)

minutes_leave_early
0.0     177291
1.0      15813
2.0       4712
4.0       4142
3.0       3978
         ...  
89.0         6
88.0         5
90.0         3
91.0         1
94.0         1
Name: count, Length: 95, dtype: int64

In [ ]:
# Since the organization has open-enrolling system, 
# meaning new students can be admitted any time of the year, 
# their total sessions are different

#Get Student's first and last join sessions
student_join_stats = (zoom.groupby('SID', as_index=False).agg(
    first_join=('Join time', 'min'),
    last_join=('Join time', 'max'),
    last_leave=('Leave time', 'max')
))

student_join_stats

,SID,first_join,last_join,last_leave
0,00000006,2025-02-20 18:25:00,2025-05-29 18:25:00,2025-05-29 19:15:00
1,00000042,2025-02-04 19:26:00,2025-05-29 19:38:00,2025-05-29 20:03:00
2,00000044,2025-02-04 20:46:00,2025-02-18 20:28:00,2025-02-18 20:31:00
3,00000077,2025-02-10 18:40:00,2025-05-29 20:26:00,2025-05-29 21:09:00
4,00000092,2025-01-17 20:04:00,2025-01-17 20:04:00,2025-01-17 20:25:00
...,...,...,...,...
1775,25000288,2025-05-15 18:31:00,2025-05-29 18:29:00,2025-05-29 19:25:00
1776,25000319,2025-04-17 19:10:00,2025-04-21 18:43:00,2025-04-21 19:25:00
1777,25000331,2025-04-21 19:52:00,2025-05-13 19:33:00,2025-05-13 19:42:00
1778,25000396,2025-05-05 18:26:00,2025-05-22 18:28:00,2025-05-22 19:25:00


# Goiten dataset cleaning

In [28]:
goiten = pd.read_excel(r"C:\Users\Admin\Documents\Studio Code\customer_retention_data\final_goiten.xlsx", dtype={'SID_8digit':str,'SID_5digit': str})
goiten.head()

,SID_full,SID_8digit,SID_5digit,Name,Grade,Call,percent_response,success_call,no_resp_calls,Successful,No_response,Match
0,1904-0019,19040019,40019,Hoàng Gia An,Grade 9,0,0.0,0,0,NaN,NaN,Grade 9-12
1,1904-0019,19040019,40019,Hoàng Gia An,EFL 69,0,0.0,0,0,NaN,NaN,Grade 9-12
2,1904-0020,19040020,40020,Hoàng Khải Lâm,Grade 9,0,0.0,0,0,NaN,NaN,Grade 9-12
3,1904-0020,19040020,40020,Hoàng Khải Lâm,EFL 69,0,0.0,0,0,NaN,NaN,Grade 9-12
4,1904-0024,19040024,40024,Lê Hà Thu,Grade 9,0,0.0,0,0,NaN,NaN,Grade 9-12


In [29]:
#fill NAs in the Successful and No_response column as 0 (representing not called on)
goiten['Successful'] = goiten['Successful'].fillna(0)
goiten['No_response'] = goiten['No_response'].fillna(0)

In [36]:
#get relative_rank: 
# since different classes have different student_call/participation levels,
# we will get the percentile of that record for that specific class to normalize
goiten['relative_rank'] = (goiten.groupby('Grade')['Call'].rank(pct=True) *100)

goiten.head()

,SID_full,SID_8digit,SID_5digit,Name,Grade,Call,percent_response,success_call,no_resp_calls,Successful,No_response,Match,relative_rank
0,1904-0019,19040019,40019,Hoàng Gia An,Grade 9,0,0.0,0,0,0,0,Grade 9-12,50.925926
1,1904-0019,19040019,40019,Hoàng Gia An,EFL 69,0,0.0,0,0,0,0,Grade 9-12,41.654357
2,1904-0020,19040020,40020,Hoàng Khải Lâm,Grade 9,0,0.0,0,0,0,0,Grade 9-12,50.925926
3,1904-0020,19040020,40020,Hoàng Khải Lâm,EFL 69,0,0.0,0,0,0,0,Grade 9-12,41.654357
4,1904-0024,19040024,40024,Lê Hà Thu,Grade 9,0,0.0,0,0,0,0,Grade 9-12,50.925926


# Canvas (quiz) dataset cleaning

In [38]:
canvas.head()

,ID,SID,Student,Section,Quiz,Score,Source_File,Quiz_Clean
0,10424,23003601,Bui Hoang Bach - 23003601,Grade 8 Algebra I F2024,1.1 Linear Functions (78170),100.0,2025-09-06T0348_Grades-Grade_8_Algebra_I_F2024,1.1 Linear Functions
1,3336,20034287,Bùi Lê Thảo Nguyên - 20034287,Grade 8 Algebra I F2024,1.1 Linear Functions (78170),NaN,2025-09-06T0348_Grades-Grade_8_Algebra_I_F2024,1.1 Linear Functions
2,9599,22001282,Bùi Trần Bảo Minh - 22001282,Grade 8 Algebra I F2024,1.1 Linear Functions (78170),100.0,2025-09-06T0348_Grades-Grade_8_Algebra_I_F2024,1.1 Linear Functions
3,2986,20043685,Bùi Vũ Gia Linh - 20043685,Grade 8 Algebra I F2024,1.1 Linear Functions (78170),100.0,2025-09-06T0348_Grades-Grade_8_Algebra_I_F2024,1.1 Linear Functions
4,10863,23000899,Cao Quỳnh Lam - 23000899,Grade 8 Algebra I F2024,1.1 Linear Functions (78170),NaN,2025-09-06T0348_Grades-Grade_8_Algebra_I_F2024,1.1 Linear Functions


In [43]:
quiz_stats = (canvas.groupby(['SID', 'Section'], as_index=False).agg(
    n_quizzess_avail=('Quiz', 'nunique'),
    n_quizzess_done=('Score', lambda x: x.notna().sum()),
    total_marks=('Score', 'sum'),
    mean_score=('Score', 'mean')
))

quiz_stats.head(10)
#the long strings SID are from the developper to test
#the emails in SIDs are from teachers/TAs

,SID,Section,n_quizzess_avail,n_quizzess_done,total_marks,mean_score
0,022935959780e30bdb2f1b2714afbb371922023c,Grade 7 Geography SP2025,24,6,0.00,0.000000
1,0a1a38604e002a657e3e774544b382a66db7defc,Grade 8 Algebra I F2024,57,11,101.68,9.243636
2,14837667e621e6f9fc6181fda599e6667e7f727c,Grade 3 Humanities F2024,22,11,0.00,0.000000
3,14d487bdf6e813c2a9b02c99cf14aae3c87349df,Grade 6 Earth Science F2024,112,6,0.00,0.000000
4,184c89d61a0a6a7b7e9a6ec9edb888e4572b4211,Grade 2 Math Spring 2025,45,6,0.00,0.000000
5,19040019,AP Biology F2024,42,39,1659.96,42.563077
6,19040019,AP Biology SP2025,44,38,1748.74,46.019474
7,19040020,AP Biology F2024,42,42,1796.79,42.780714
8,19040020,AP Biology SP2025,44,44,2655.88,60.360909
9,19040024,AP Biology F2024,42,35,1506.10,43.031429


In [49]:
#get avg_score2_all_quizzes: total marks divided all 
quiz_stats['avg_score1_all_quizzes'] = (
    quiz_stats['total_marks'] / quiz_stats['n_quizzess_avail']
)

#because in some class, there are quizzes that are optional 
#so students rarely do them -> we cannot take total marks / total_quizzes (a lot will be nulls)
#get avg_score2_done_quizzes: total marks / the quizzes done only
quiz_stats['avg_score2_done_quizzes'] = (
    quiz_stats['total_marks'] / quiz_stats['n_quizzess_done']
)

#because some classes have different quiz scale (e.g., 100 vs 20)
#get percentile of that student's score compared to the rest in that class (normalize)
quiz_stats['score_percentile1'] = (
    quiz_stats.groupby('Section')['avg_score1_all_quizzes']
    .rank(pct=True)*100
)

quiz_stats['score_percentile2'] = (
    quiz_stats.groupby('Section')['avg_score2_done_quizzes']
    .rank(pct=True)*100
)

#get completion_rate
quiz_stats['percent_completion'] = (
    quiz_stats['n_quizzess_done'] / quiz_stats['n_quizzess_avail'] 
)*100

quiz_stats.head(10)

,SID,Section,n_quizzess_avail,n_quizzess_done,total_marks,mean_score,avg_score1_all_quizzes,avg_score2_done_quizzes,score_percentile1,score_percentile2,percent_completion
0,022935959780e30bdb2f1b2714afbb371922023c,Grade 7 Geography SP2025,24,6,0.00,0.000000,0.000000,0.000000,12.681159,12.681159,25.000000
1,0a1a38604e002a657e3e774544b382a66db7defc,Grade 8 Algebra I F2024,57,11,101.68,9.243636,1.783860,9.243636,41.269841,41.269841,19.298246
2,14837667e621e6f9fc6181fda599e6667e7f727c,Grade 3 Humanities F2024,22,11,0.00,0.000000,0.000000,0.000000,12.931034,12.931034,50.000000
3,14d487bdf6e813c2a9b02c99cf14aae3c87349df,Grade 6 Earth Science F2024,112,6,0.00,0.000000,0.000000,0.000000,16.250000,16.250000,5.357143
4,184c89d61a0a6a7b7e9a6ec9edb888e4572b4211,Grade 2 Math Spring 2025,45,6,0.00,0.000000,0.000000,0.000000,10.663507,10.663507,13.333333
5,19040019,AP Biology F2024,42,39,1659.96,42.563077,39.522857,42.563077,84.571429,71.428571,92.857143
6,19040019,AP Biology SP2025,44,38,1748.74,46.019474,39.744091,46.019474,88.028169,81.690141,86.363636
7,19040020,AP Biology F2024,42,42,1796.79,42.780714,42.780714,42.780714,96.571429,73.142857,100.000000
8,19040020,AP Biology SP2025,44,44,2655.88,60.360909,60.360909,60.360909,98.591549,97.887324,100.000000
9,19040024,AP Biology F2024,42,35,1506.10,43.031429,35.859524,43.031429,74.285714,73.714286,83.333333


# Aggregrate metrics, grouped by SIDs